In [135]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Users/pavelstasevsky/anaconda/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test', 'poly', 'fmin']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


3. Предобработка для номинальных:
* one-hot encoding (? можно попробовать без него)
4. Разделение на обучающую и валидационную
* На обучающей аккуратный рассчет SLH
* Добавление спрямляющих пространств (?)
* Обучение моделей (LR, RF, XGB, ET, NN)
* Проверка на валидационной
* Объединение моделей в ансамбль

# 1. Загрузка данных

In [136]:
#train = pd.read_csv('./data/train.csv', header=0, sep=';')
#test = pd.read_csv('./data/test.csv', header=0, sep=';')
train = pd.read_csv('./data/train_fe_2806.csv', header=0, sep=';')
test = pd.read_csv('./data/test_fe_2806.csv', header=0, sep=';')

In [137]:
print train.shape
print test.shape

(70000, 26)
(30000, 25)


In [138]:
train.head().T

,0,1,2,3,4
id,0.000000,1.000000,2.000000,3.000000,4.000000
age,18393.000000,20228.000000,18857.000000,17623.000000,17474.000000
gender,2.000000,1.000000,1.000000,2.000000,1.000000
height,168.000000,156.000000,165.000000,169.000000,156.000000
weight,62.000000,85.000000,64.000000,82.000000,56.000000
ap_hi,110.000000,140.000000,130.000000,150.000000,100.000000
ap_lo,80.000000,90.000000,70.000000,100.000000,60.000000
cholesterol,1.000000,3.000000,3.000000,1.000000,1.000000
gluc,1.000000,1.000000,1.000000,1.000000,1.000000
smoke,0.000000,0.000000,0.000000,0.000000,0.000000


# 2. Подготовка данных

Признаки, которые мы будем использовать для обучения (пока, как есть).

In [139]:
columns_numeric = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'age_year', 'BMI', 'BMI_Breitman', 'risk_cd']
columns_binary = ['age_over_60', 'age_less_40', 'weight_over_90', 'weight_male_over_100', 
           'weight_female_over_100', 'ap_hi_over_140', 'ap_lo_over_95', 'BMI_over_35' , 'alco', 'active', 'smoke']
columns_to_onehotenc = ['cholesterol', 'gender', 'gluc', 'cluster']
#columns = ['age', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol']

Выбираем только те признаки, которые будем дальше использовать.

Для номинальных признаков со множеством значений делаем one hot encoding.

In [140]:
import itertools

train_y = train['cardio']
train_x = train[list(itertools.chain(columns_numeric, columns_binary, columns_to_onehotenc))]
test_x = test[list(itertools.chain(columns_numeric, columns_binary, columns_to_onehotenc))]

Для числовых признаков делаем RobustScale.

In [141]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
train_x[columns_numeric] = scaler.fit_transform(train_x[columns_numeric])
test_x[columns_numeric] = scaler.transform(test_x[columns_numeric])

/Users/pavelstasevsky/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/pavelstasevsky/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Для номинальных фич со множеством значений применяем one_hot кодирование.

In [142]:
def one_hot(df, cols):
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df.drop(each, axis=1, inplace=True)
    return df

train_x = one_hot(train_x, columns_to_onehotenc)
test_x = one_hot(test_x, columns_to_onehotenc)

In [143]:
train_x.head()

,age,height,weight,ap_hi,ap_lo,age_year,BMI,BMI_Breitman,risk_cd,age_over_60,...,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12
0,-0.357630,0.272727,-0.588235,-0.5,0.0,-0.357630,-0.693720,-0.711765,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.143325,-0.818182,0.764706,1.0,1.0,0.143325,1.348243,1.135294,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.230958,0.000000,-0.470588,0.5,-1.0,-0.230958,-0.450982,-0.470588,-1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.567841,0.363636,0.588235,1.5,2.0,-0.567841,0.368710,0.423529,-1.0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,-0.608518,-0.818182,-0.941176,-1.0,-2.0,-0.608518,-0.529227,-0.570588,-1.0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Разделим обучающую выборку на обучающую и валидационную (10%).

In [144]:
X_train, X_validation, y_train, y_validation = train_test_split(train_x, train['cardio'],
                                                                test_size=0.10, random_state=111)

# 3. Обучение моделей

1. Разобьем train на три фолда
2. На двух посчитаем SL и заполним третий (и так три раза)
3. Обучаем модель три раза, смотрим на качество
4. Обучаем итоговую модель: считаем SL по всему тесту, для validation и test заполняем по тесту
5. Проверяем значения log_loss на validation
6. Прогнозируем для test

In [145]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [146]:
def smoothed_likelyhood(x_train, x_test, y_train, k=10):
    
    x_train['label'] = y_train
    # вычисляем глобальное среднее у
    global_mean = x_train['label'].mean()
    
    # получаем список всех признаков
    columns =  list(x_train.columns.values)
    columns.remove('label')
    for column in columns:
        print column
        # получаем набор уникальных значений для признака
        uniques = x_train[column].unique()
        
        # теперь для всех уникальных значений считаем SL
        sl_uv = {}
        for unique in uniques:
            sl = (x_train[x_train[column] == unique][column].count()*x_train[x_train[column] == unique]['label'].mean() + global_mean*k)/(x_train[x_train[column] == unique][column].count() + k)
            sl_uv[unique] = sl
        
        # добавляем новый столбец в x_train
        x_train[column+'_SL'] = x_train[column].apply(lambda x: sl_uv[x])
        
        #добавляем новый столбец в x_test
        x_test[column+'_SL'] = x_test[column].apply(lambda x: sl_uv[x] if x in sl_uv.keys() else global_mean)
        #print 'Column SL: ', x_test[column+'_SL']
        
    return x_train.drop('label', axis=1), x_test

## 3.1 LogisticRegression

In [147]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from hyperopt import hp, fmin, Trials, tpe, STATUS_OK

In [72]:
poly = PolynomialFeatures(2)

poly_x_train = poly.fit_transform(X_train)

In [73]:
poly_x_validation = poly.transform(X_validation)
poly_x_test = poly.transform(test_x)

In [74]:
print poly_x_train.shape

(63000, 903)


In [77]:
def LR_learn(random_state=1):
    def hyperopt_rf(params):
        print "Training with params : "
        print params
        clf = LogisticRegression(**params)
        score = cross_val_score(clf, poly_x_train, y_train, scoring='neg_log_loss', cv=3).mean()
        print "Score: ", score
        return score

    def f(params):
        log_loss = hyperopt_rf(params)
        return {'loss': -log_loss, 'status': STATUS_OK}

    space4lr = {
        'C': hp.uniform('C', 0, 1),
        'penalty': hp.choice('penalty', ['l1', 'l2']),
        'random_state': random_state,
        'n_jobs': 4

    }

    trials = Trials()
    best = fmin(f, space4lr, algo=tpe.suggest, max_evals=15, trials=trials)
    print 'best:'
    print best
    return best

Обучение первой модели с random_state=11.

In [79]:
best_lr_1 = LR_learn(random_state=11)

Training with params : 
{'penalty': 'l2', 'C': 0.21022626810031264, 'random_state': 11, 'n_jobs': 4}
Score:  -0.544431902892
Training with params : 
{'penalty': 'l1', 'C': 0.7120726238129801, 'random_state': 11, 'n_jobs': 4}
Score:  -0.544022046947
Training with params : 
{'penalty': 'l1', 'C': 0.5713993065019433, 'random_state': 11, 'n_jobs': 4}
Score:  -0.543771349619
Training with params : 
{'penalty': 'l1', 'C': 0.8043616430748548, 'random_state': 11, 'n_jobs': 4}
Score:  -0.544145957401
Training with params : 
{'penalty': 'l1', 'C': 0.7370026146229589, 'random_state': 11, 'n_jobs': 4}
Score:  -0.544054702401
Training with params : 
{'penalty': 'l2', 'C': 0.14748957769581927, 'random_state': 11, 'n_jobs': 4}
Score:  -0.544222804881
Training with params : 
{'penalty': 'l2', 'C': 0.7916367399679635, 'random_state': 11, 'n_jobs': 4}
Score:  -0.545159076383
Training with params : 
{'penalty': 'l1', 'C': 0.6226356491652284, 'random_state': 11, 'n_jobs': 4}
Score:  -0.5438691613
Training

Проверка на валидации.

In [80]:
penalty = ['l1', 'l2']
random_state=11
lr1 = LogisticRegression(C=best_lr_1['C'], penalty=penalty[best_lr_1['penalty']], random_state=random_state)
lr1.fit(poly_x_train, y_train)

predict_validation_lr1 = lr1.predict_proba(poly_x_validation)
print log_loss(y_validation, predict_validation_lr1)

0.546570133575


Прогнозируем тест.

In [81]:
predict_test_lr1 = lr1.predict_proba(poly_x_test)

In [82]:
predict_test_lr1

array([[ 0.61580515,  0.38419485],
       [ 0.65883644,  0.34116356],
       [ 0.57706703,  0.42293297],
       ..., 
       [ 0.61727601,  0.38272399],
       [ 0.83648859,  0.16351141],
       [ 0.68080948,  0.31919052]])

Еще одна модель с LR и random_state=117

In [83]:
best_lr_2 = LR_learn(random_state=117)

Training with params : 
{'penalty': 'l2', 'C': 0.17262339689219697, 'random_state': 117, 'n_jobs': 4}
Score:  -0.544316229205
Training with params : 
{'penalty': 'l2', 'C': 0.7140315295603211, 'random_state': 117, 'n_jobs': 4}
Score:  -0.545114379167
Training with params : 
{'penalty': 'l1', 'C': 0.5066387713612533, 'random_state': 117, 'n_jobs': 4}
Score:  -0.543635746678
Training with params : 
{'penalty': 'l2', 'C': 0.27131506734516453, 'random_state': 117, 'n_jobs': 4}
Score:  -0.544581500929
Training with params : 
{'penalty': 'l1', 'C': 0.03820732436756635, 'random_state': 117, 'n_jobs': 4}
Score:  -0.543922652206
Training with params : 
{'penalty': 'l2', 'C': 0.03476000472365792, 'random_state': 117, 'n_jobs': 4}
Score:  -0.543826002028
Training with params : 
{'penalty': 'l2', 'C': 0.4688464683629784, 'random_state': 117, 'n_jobs': 4}
Score:  -0.544896857461
Training with params : 
{'penalty': 'l1', 'C': 0.8918325081056311, 'random_state': 117, 'n_jobs': 4}
Score:  -0.544266601

In [84]:
random_state=117
lr2 = LogisticRegression(C=best_lr_1['C'], penalty=penalty[best_lr_1['penalty']], random_state=random_state)
lr2.fit(poly_x_train, y_train)

predict_validation_lr2 = lr2.predict_proba(poly_x_validation)
print log_loss(y_validation, predict_validation_lr1)

0.546570133575


In [85]:
predict_test_lr2 = lr2.predict_proba(poly_x_test)

## 3.2 RandomForest

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [94]:
def RF_learn(random_state=1):
    def hyperopt_rf(params):
        print "Training with params : "
        print params
        clf = RandomForestClassifier(**params)
        score = cross_val_score(clf, X_train, y_train, scoring='neg_log_loss', cv=3).mean()
        print "Score: ", score
        return score

    def f(params):
        acc = hyperopt_rf(params)
        return {'loss': -acc, 'status': STATUS_OK}

    space4rf = {
        'n_estimators': hp.choice('n_estimators', range(50,300)),
        'max_depth': hp.choice('max_depth', range(1,30)),
        'max_features': hp.choice('max_features', range(5,25)),
        'criterion': hp.choice('criterion', ["gini", "entropy"]),
        'random_state': random_state,
        'n_jobs': 4

    }

    trials = Trials()
    best = fmin(f, space4rf, algo=tpe.suggest, max_evals=100, trials=trials)
    print 'best:'
    print best
    return best

In [95]:
best_rf_1 = RF_learn(random_state=11)

Training with params : 
{'n_estimators': 232, 'n_jobs': 4, 'criterion': 'gini', 'max_features': 23, 'random_state': 11, 'max_depth': 28}
Score:  -0.575379458852
Training with params : 
{'n_estimators': 297, 'n_jobs': 4, 'criterion': 'entropy', 'max_features': 19, 'random_state': 11, 'max_depth': 14}
Score:  -0.544326839464
Training with params : 
{'n_estimators': 170, 'n_jobs': 4, 'criterion': 'entropy', 'max_features': 5, 'random_state': 11, 'max_depth': 20}
Score:  -0.549865579497
Training with params : 
{'n_estimators': 295, 'n_jobs': 4, 'criterion': 'entropy', 'max_features': 17, 'random_state': 11, 'max_depth': 12}
Score:  -0.541724623137
Training with params : 
{'n_estimators': 89, 'n_jobs': 4, 'criterion': 'entropy', 'max_features': 21, 'random_state': 11, 'max_depth': 4}
Score:  -0.549943561388
Training with params : 
{'n_estimators': 178, 'n_jobs': 4, 'criterion': 'entropy', 'max_features': 11, 'random_state': 11, 'max_depth': 5}
Score:  -0.548420978526
Training with params : 

Проверка на валидации.

In [98]:
criterion = ["gini", "entropy"]
rf1 = RandomForestClassifier(max_features=best_rf_1['max_features'], n_estimators=best_rf_1['n_estimators'], 
                            criterion=criterion[best_rf_1['criterion']], max_depth=best_rf_1['max_depth'], 
                            random_state=11, n_jobs=4)

rf1.fit(X_train, y_train)

predict_validation_rf1 = rf1.predict_proba(X_validation)
print log_loss(y_validation, predict)

0.546638611852


Прогнозируем тест.

In [100]:
predict_test_rf1 = rf1.predict_proba(test_x)

In [102]:
criterion = ["gini", "entropy"]
rf2 = RandomForestClassifier(max_features=12, n_estimators=272, 
                            criterion='gini', max_depth=15, 
                            random_state=117, n_jobs=4)

rf2.fit(X_train, y_train)

predict_validation_rf2 = rf2.predict_proba(X_validation)
print log_loss(y_validation, predict)

0.546638611852


In [103]:
predict_test_rf2 = rf2.predict_proba(test_x)

## 3.4 XgBoost

In [148]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

In [150]:
def XGB_learn(random_state=1):
    def hyperopt_xgb(params):
        print "Training with params : "
        print params
        clf = XGBClassifier(**params)
        score = cross_val_score(clf, X_train, y_train, scoring='neg_log_loss', cv=3).mean()
        print "Score: ", score
        return score

    def f(params):
        acc = hyperopt_xgb(params)
        return {'loss': -acc, 'status': STATUS_OK}

    random_state= 11
    space4xgb = {
        'n_estimators': hp.choice('n_estimators', range(50,150)),
        'max_depth': hp.choice('max_depth', range(1,7)),
        'learning_rate': hp.uniform('learning_rate', 0, 1),
        'min_child_weight': hp.choice('min_child_weight', range(10,20)),
        'reg_alpha': hp.uniform('reg_alpha', 1, 3),
        'subsample': hp.uniform('subsample', 0, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 1),
        'objective': 'binary:logistic',
        'seed': random_state,
        'nthread': 4

    }

    trials = Trials()
    best = fmin(f, space4xgb, algo=tpe.suggest, max_evals=300, trials=trials)
    print 'best:'
    print best
    return best

In [151]:
best_xgb_1 = XGB_learn(random_state=114)

Training with params : 
{'reg_alpha': 1.8403617677586532, 'colsample_bytree': 0.6377915471372693, 'learning_rate': 0.7023423477608088, 'nthread': 4, 'min_child_weight': 14, 'n_estimators': 95, 'subsample': 0.02511793836213727, 'seed': 11, 'objective': 'binary:logistic', 'max_depth': 5}
Score:  -0.67151048597
Training with params : 
{'reg_alpha': 1.666608883566014, 'colsample_bytree': 0.2207997747667971, 'learning_rate': 0.49359928020365107, 'nthread': 4, 'min_child_weight': 14, 'n_estimators': 73, 'subsample': 0.7729582269713277, 'seed': 11, 'objective': 'binary:logistic', 'max_depth': 2}
Score:  -0.542477422207
Training with params : 
{'reg_alpha': 2.9204441528119665, 'colsample_bytree': 0.2962429942966015, 'learning_rate': 0.40577683989152735, 'nthread': 4, 'min_child_weight': 11, 'n_estimators': 139, 'subsample': 0.40504384793562076, 'seed': 11, 'objective': 'binary:logistic', 'max_depth': 3}
Score:  -0.54585084952
Training with params : 
{'reg_alpha': 1.270727040361451, 'colsample_

In [152]:
xgb1 = XGBClassifier(reg_alpha=best_xgb_1['reg_alpha'], colsample_bytree=best_xgb_1['colsample_bytree'], 
                    learning_rate=best_xgb_1['learning_rate'], nthread=4, min_child_weight=best_xgb_1['min_child_weight'],
                   n_estimators=best_xgb_1['n_estimators'], subsample=best_xgb_1['subsample'], seed=114, 
                   objective='binary:logistic', max_depth=best_xgb_1['max_depth'])
#{'reg_alpha': 2.8125709367673073, 'colsample_bytree': 0.913818245469191, 'learning_rate': 0.06905574617173077, 
# 'min_child_weight': 9, 'n_estimators': 84, 'subsample': 0.6465149185565322, 'max_depth': 3}
xgb1.fit(X_train, y_train)

predict_validation_xgb1 = xgb1.predict_proba(X_validation)
print log_loss(y_validation, predict_validation_xgb1)

0.543102587373


Прогнозируем тест.

In [153]:
predict_test_xgb1 = xgb1.predict_proba(test_x)

Вторая модель.

In [154]:
xgb2 = XGBClassifier(reg_alpha=2.694563437569389, colsample_bytree=0.8768335778370171, 
                    learning_rate=0.062452511506258065, nthread=4, min_child_weight=14,
                   n_estimators=137, subsample=0.6914283064766742, seed=115, 
                   objective='binary:logistic', max_depth=4)

xgb2.fit(X_train, y_train)

predict_validation_xgb2 = xgb2.predict_proba(X_validation)
print log_loss(y_validation, predict_validation_xgb2)

0.541695628415


In [155]:
predict_test_xgb2 = xgb2.predict_proba(test_x)

Третья модель.

In [157]:
xgb3 = XGBClassifier(reg_alpha=2.3248532256940355, colsample_bytree=0.928325758550966, 
                    learning_rate=0.10555416217450769, nthread=4, min_child_weight=19,
                   n_estimators=85, subsample=0.8594437437833493, seed=114, 
                   objective='binary:logistic', max_depth=5)


xgb3.fit(X_train, y_train)

predict_validation_xgb3 = xgb3.predict_proba(X_validation)
print log_loss(y_validation, predict_validation_xgb3)

0.541734756713


In [158]:
predict_test_xgb3 = xgb3.predict_proba(test_x)

Четвертая модель.

In [162]:
xgb4 = XGBClassifier(reg_alpha=2.8140796108679527, colsample_bytree=0.8358236357427814, 
                    learning_rate=0.04469329599940971, nthread=4, min_child_weight=17,
                   n_estimators=130, subsample=0.5671630321375755, seed=118, 
                   objective='binary:logistic', max_depth=4)

xgb4.fit(X_train, y_train)

predict_validation_xgb4 = xgb4.predict_proba(X_validation)
print log_loss(y_validation, predict_validation_xgb4)

0.54201328085


In [163]:
predict_test_xgb4 = xgb4.predict_proba(test_x)

Пятая модель.

In [164]:
xgb5 = XGBClassifier(reg_alpha=2.8432031545028416, colsample_bytree=0.8947122406701733, 
                    learning_rate=0.07547000831742193, nthread=4, min_child_weight=19,
                   n_estimators=125, subsample=0.6590164633486921, seed=119, 
                   objective='binary:logistic', max_depth=4)
xgb5.fit(X_train, y_train)

predict_validation_xgb5 = xgb5.predict_proba(X_validation)
print log_loss(y_validation, predict_validation_xgb5)

0.541807529242


In [165]:
predict_test_xgb5 = xgb5.predict_proba(test_x)

# 4. Mix моделей

## 4.1 Проверка на валидации

Сначала проверяем на валидационной выборке.

In [166]:
validation_mix = pd.DataFrame(predict_validation_xgb1[:,1])
validation_mix.columns = ['xgb1']

In [167]:
validation_mix['xgb2'] = predict_validation_xgb2[:,1]
validation_mix['xgb3'] = predict_validation_xgb3[:,1]
validation_mix['xgb4'] = predict_validation_xgb4[:,1]
validation_mix['xgb5'] = predict_validation_xgb5[:,1]

In [168]:
validation_mix['mean_all'] = (validation_mix.xgb1 + validation_mix.xgb2+ validation_mix.xgb3+validation_mix.xgb4+validation_mix.xgb5)/5

In [169]:
print 'Log loss all', log_loss(y_validation, validation_mix['mean_all'])

Log loss all 0.541571399437


In [170]:
validation_mix['mean_colsample'] = (validation_mix.xgb1*0.914 + validation_mix.xgb2*0.877+ validation_mix.xgb3*0.928+validation_mix.xgb4*0.836+validation_mix.xgb5*0.895)/(0.914+0.877+0.928+0.836+0.895)

In [171]:
print 'Log loss xgb', log_loss(y_validation, validation_mix['mean_colsample'])

Log loss xgb 0.541566762924


In [172]:
validation_mix['mean_xgb_2_5'] = (validation_mix.xgb2*0.877+ validation_mix.xgb3*0.928+validation_mix.xgb4*0.836+validation_mix.xgb5*0.895)/(0.877+0.928+0.836+0.895)

In [173]:
print 'Log loss xgb', log_loss(y_validation, validation_mix['mean_xgb_2_5'])

Log loss xgb 0.541475115664


## 4.2 Предсказание для теста

In [175]:
test_mix = pd.DataFrame(predict_test_xgb1[:,1])
test_mix.columns = ['xgb1']

test_mix['xgb2'] = predict_test_xgb2[:,1]
test_mix['xgb3'] = predict_test_xgb3[:,1]
test_mix['xgb4'] = predict_test_xgb4[:,1]
test_mix['xgb5'] = predict_test_xgb5[:,1]

Сохраняем в файл.

In [180]:
test_mix.to_csv('./result/result_models_3006.csv', header=True, index=False, sep=';')

In [181]:
test_mix['mean_all'] = (test_mix.xgb1 + test_mix.xgb2+ test_mix.xgb3+test_mix.xgb4+test_mix.xgb5)/5

In [182]:
test_mix['mean_collsample'] = (test_mix.xgb1*0.914 + test_mix.xgb2*0.877+ test_mix.xgb3*0.928+test_mix.xgb4*0.836+test_mix.xgb5*0.895)/(0.914+0.877+0.928+0.836+0.895)

In [183]:
test_mix['mean_xgb_2_5'] = (test_mix.xgb2*0.877+ test_mix.xgb3*0.928+test_mix.xgb4*0.836+test_mix.xgb5*0.895)/(0.877+0.928+0.836+0.895)

In [184]:
test_mix['mean_all'].to_csv('./result/res12_all_3006.csv', index=False, header=None)

In [185]:
test_mix['mean_collsample'].to_csv('./result/res13_collsample_3006.csv', index=False, header=None)

In [186]:
test_mix['mean_xgb_2_5'].to_csv('./result/res14_xgb_25_3006.csv', index=False, header=None)